In [ ]:
# Importamos las dependencias necesarias.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import sklearn_pandas
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn_pandas import DataFrameMapper

In [ ]:
# Importamos el dataset.
dataset = pd.read_csv('TravelInsurancePrediction.csv')

# Renombramos las variables.
dataset = dataset.rename(columns={'Idx':'idx', 'Age':'age', 'Employment Type':'employment_type', 'GraduateOrNot':'graduate_or_not', 'AnnualIncome':'annual_income', 'FamilyMembers':'family_members', 'ChronicDiseases':'chronic_diseases', 'EverTravelledAbroad':'ever_travelled_abroad', 'FrequentFlyer':'frequent_flyer', 'TravelInsurance':'travel_insurance'})

# Seleccionamos la primer columna 'idx', como índice.
dataset = dataset.set_index("idx")

# Reemplazamos los valores 'Yes' y 'No' por 0 y 1 de las variables indicadas en el tp1.
dataset["graduate_or_not"] = dataset.graduate_or_not.replace(['No', 'Yes'], [0,1])
dataset["frequent_flyer"] = dataset.frequent_flyer.replace(['No', 'Yes'], [0,1])
dataset["ever_travelled_abroad"] = dataset.ever_travelled_abroad.replace(['No', 'Yes'], [0,1])

# Reemplazamos los dos posibles valores de la columna 'employment_type' por 0 y 1.
dataset["employment_type"] = dataset.employment_type.replace(['Private Sector/Self Employed', 'Government Sector'], [0,1])

# Dividimos el dataset en train (60%), test (20%) y validation (20%)
train, not_train = train_test_split(dataset, test_size=0.4, random_state=42)
validation, test = train_test_split(not_train, test_size=0.5, random_state=42)

In [ ]:
dataset.sample(10)

In [ ]:
# Escalamos las variables numéricas.
mapper = DataFrameMapper([
    (['age'],[StandardScaler()]),
    #(['employment_type'],[]),
    #(['graduate_or_not'],[]),
    (['annual_income'],[StandardScaler()]),
    (['family_members'],[StandardScaler()])
    #(['chronic_diseases'],[]),
    #(['frequent_flyer'],[]),
    #(['ever_travelled_abroad'],[]),
    #(['travel_insurance'],[])
])

mapper.fit(train)
dataset2 = pd.DataFrame(mapper.transform(dataset), columns=['age', 'annual_income', 'family_members'])
dataset2